In [4]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [5]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        if d_type=='train':
            with open('../../data/chat_feature_train.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='val':
            with open('../../data/chat_feature_val.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='test':
            with open('../../data/chat_feature_test.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        
        with open('../../data/video_statistic_features_one2.pickle',"rb") as f2:  
            self.image_result=pickle.load(f2)
        with open('../../data/audio_entropy_emd.pickle',"rb") as f3:  
            self.audio_result=pickle.load(f3)
        
        with open('../../label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.chat_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.chat_result[game_id]):
                    s_window+=((self.chat_result[game_id][vframe+idx]))#vframe의 chat
                    s_window+=list(self.audio_result[game_id][(vframe+idx)*10:(vframe+idx+1)*10])#vframe의 audio
#                     s_window+=[(self.image_result[game_id][vframe+idx])]#vframe의 image
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*138
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [6]:
train=Mul_data('train')
val=Mul_data('val')
print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[-4.23758756e-03,  8.49383231e-03,  1.44858090e-02, ...,
        -3.37451883e-03,  3.59619475e-03, -4.46354486e-03],
       [ 1.31821318e-04,  3.67566757e-03,  1.84836853e-02, ...,
        -6.83272836e-03, -1.36067919e-02, -1.21897105e-02],
       [ 2.78287055e-03, -2.53155967e-03,  2.11640801e-02, ...,
        -5.03458177e-03, -1.22952435e-03,  4.21928000e-03],
       ...,
       [-9.56570916e-03,  2.41865851e-02,  5.35075879e-03, ...,
        -4.22192635e-04,  1.91354366e-03,  3.05473938e-03],
       [-8.14033020e-03,  2.75277682e-02,  2.89613754e-03, ...,
         6.71383702e-04, -1.62371441e-04,  4.06735896e-05],
       [-7.51412148e-03,  2.00215783e-02,  8.99404753e-03, ...,
         5.35640891e-03, -2.38230451e-03, -1.40516128e-02]]), 0)


In [13]:
input_size=138
hidden_size=23
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        feature,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        out = self._lin(feature[:,-1,:])
        return out

In [13]:
a,b,c=train[100]

c

0

In [14]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
weight_dir='./2features_ent_emd+채팅_undersampling/'
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(200):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.6932000517845154,p 0.2112332773655419, r 0.8609271523178808, f 0.33923367981559605

1
epoch 1 train_loss : 0 , val_loss : 0.6636262536048889,p 0, r 0, f 0

2
epoch 2 train_loss : 0 , val_loss : 0.742064356803894,p 0, r 0, f 0

3
epoch 3 train_loss : 0 , val_loss : 0.6776425242424011,p 0.3147490643892141, r 0.7240618101545254, f 0.43876663768309815

4
epoch 4 train_loss : 0 , val_loss : 0.6194946765899658,p 0.35064184110095986, r 0.6693156732891832, f 0.4601957957046368

5
epoch 5 train_loss : 0 , val_loss : 0.5641056895256042,p 0.36751921380874386, r 0.6439293598233996, f 0.46795540226197163

6
epoch 6 train_loss : 0 , val_loss : 0.491534024477005,p 0.45663213487710747, r 0.49624724061810155, f 0.4756162064952925

7
epoch 7 train_loss : 0 , val_loss : 0.6315155625343323,p 0.3180446002531892, r 0.720971302428256, f 0.4413811744036759

8
epoch 8 train_loss : 0 , val_loss : 0.7131094932556152,p 0.2824675324675325, r 0.7874172185430464, f 0.415782725

epoch 67 train_loss : 0 , val_loss : 0.5582074522972107,p 0.3937899786780384, r 0.652317880794702, f 0.49110852584344356

68
epoch 68 train_loss : 0 , val_loss : 0.4878767132759094,p 0.4407763769077638, r 0.5865342163355408, f 0.5033150217844289

69
epoch 69 train_loss : 0 , val_loss : 0.5745502710342407,p 0.36204498029214005, r 0.6894039735099338, f 0.47476436606871386

70
epoch 70 train_loss : 0 , val_loss : 0.5679855942726135,p 0.390812162695801, r 0.6554083885209713, f 0.4896511915560321

71
epoch 71 train_loss : 0 , val_loss : 0.723745584487915,p 0.28520541439088104, r 0.795364238410596, f 0.4198566684146129

72
epoch 72 train_loss : 0 , val_loss : 0.5541501045227051,p 0.3975984889368592, r 0.6505518763796909, f 0.49355216881594377

73
epoch 73 train_loss : 0 , val_loss : 0.5412799119949341,p 0.40497360377882746, r 0.6434878587196468, f 0.49710095497953616

74
epoch 74 train_loss : 0 , val_loss : 0.48148202896118164,p 0.4839421918908069, r 0.5322295805739514, f 0.5069386038687973


In [15]:
weight_dir='./2features_ent_emd+채팅_undersampling/'

test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 22 10 0 tensor(22) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 27 5 tensor(0) tensor(5)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 16 16 tensor(0) tensor(16)
0 0 15 17 tensor(0) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 20 12 tensor(0) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 3 22 0 tensor(10) tensor(7)
3 0 14 15 tensor(3) tensor(18)
0 0 24 8 tensor(0) tensor(8)
0 0 32 0 tensor(0) tensor(0)
0 12 7 13 tensor(1

0 0 32 0 tensor(0) tensor(0)
0 2 30 0 tensor(2) tensor(0)
1 0 15 16 tensor(1) tensor(17)
3 0 19 10 tensor(3) tensor(13)
0 0 22 10 tensor(0) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 15 17 0 tensor(15) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 18 14 tensor(0) tensor(14)
0 9 12 11 tensor(9) tensor(11)
10 22 0 0 tensor(32) tensor(10)
2 16 14 0 tensor(18) tensor(2)
0 9 23 0 tensor(9) tensor(0)
0 15 17 0 tensor(15) tensor(0)
0 0 32 0 tensor(0) tensor(0)
15 7 10 0 tensor(22) tensor(15)
8 0 23 1 tensor(8) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 20 9 0 tensor(23) tensor(3)
27 0 0 5 tensor(27) tensor(32)
5 0 12 15 tensor(5) tensor(20)
0 4 28 0 tensor(4) tensor(0)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
23 0 3 6 tensor(23) tensor(29)
1 0 20 11 tensor(1) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 t

0 0 32 0 tensor(0) tensor(0)
0 9 14 9 tensor(9) tensor(9)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 6 22 4 tensor(6) tensor(4)
0 32 0 0 tensor(32) tensor(0)
10 22 0 0 tensor(32) tensor(10)
8 5 15 4 tensor(13) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 27 5 0 tensor(27) tensor(0)
0 3 29 0 tensor(3) tensor(0)
0 0 29 3 tensor(0) tensor(3)
0 0 9 23 tensor(0) tensor(23)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 25 7 tensor(0) tensor(7)
0 0 26 6 tensor(0) tensor(6)
0 0 20 12 tensor(0) tensor(12)
0 0 12 20 tensor(0) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 23 9 tensor(0) tensor(9)
6 2 23 1 tensor(8) tensor(7)
0 6 24 2 tensor(6) tensor(2)
0 0 10 22 tensor(0) tensor(22)
12 0 6 14 tensor(12) tensor(26)
8 3 21 0 tensor(11) tensor(8)
23 9 0 0 tensor(32) tensor(23)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) te

23 4 5 0 tensor(27) tensor(23)
13 10 8 1 tensor(23) tensor(14)
0 0 26 6 tensor(0) tensor(6)
7 0 25 0 tensor(7) tensor(7)
29 3 0 0 tensor(32) tensor(29)
0 3 29 0 tensor(3) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
22 5 5 0 tensor(27) tensor(22)
7 16 9 0 tensor(23) tensor(7)
0 0 32 0 tensor(0) tensor(0)
0 0 26 6 tensor(0) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 20 1 tensor(11) tensor(12)
21 0 11 0 tensor(21) tensor(21)
0 0 24 8 tensor(0) tensor(8)
0 0 32 0 tensor(0) tensor(0)
14 0 16 2 tensor(14) tensor(16)
26 0 6 0 tensor(26) tensor(26)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
22 2 8 0 tensor(24) tensor(22)
26 0 3 3 tensor(26) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0)

In [11]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [16]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('../../label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6064139941690962, recall : 0.5459317585301837, f1 : 0.574585635359116, accuracy : 0.875
102844341902586509
precision : 0.4528301886792453, recall : 0.8537549407114624, f1 : 0.5917808219178082, accuracy : 0.8606173994387278
102844401152267937
precision : 0.23787528868360278, recall : 0.2562189054726368, f1 : 0.24670658682634733, accuracy : 0.7252075141983398
102844212430927059
precision : 0.5521936459909228, recall : 0.553030303030303, f1 : 0.5526116578349736, accuracy : 0.844596371285827
102844412708953395
precision : 0.4627906976744186, recall : 0.7210144927536232, f1 : 0.5637393767705383, accuracy : 0.8501945525291829
102844212429944013
precision : 0.522673031026253, recall : 0.5902964959568733, f1 : 0.5544303797468355, accuracy : 0.8279569892473119
102844341912679064
precision : 0.6486486486486487, recall : 0.4247787610619469, f1 : 0.5133689839572193, accuracy : 0.9061855670103093
102844235753749959
precision : 0.254278728606357, recall : 0.264631043

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571

In [22]:
weight_dir='./2features_ent_emd+채팅_undersampling/'
test=Mul_data('train')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output=model(inputs)
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                print(g)
                result[g]=[output[idx].tolist()]
            else:
                result[g].append(output[idx].tolist())


data load fin
102844412722519367
102844212429550795
102844401151219358
102844401154430631
102844412717014335
102844401153971877
102844224148503678
102844412722847048
102844401152857762
102844412707380528
102844212431516886
102844283027925085
102844412716227901
102844412710001974
102844294670878922
102844294670551241
102844283023599703
102844412704496937
102844235751783874
102844401152071328
102844412709674293
102844401153447587
102844224148896895
102844235746868664
102979081290790284
102844283027531868
102844212431975640
102844401155937960
102844212429092040
102844341906649746
102844412706987311
102844412721339716
102844212430402768
102844341905011343
102844235753356742
102844235750997440
102844412709346612
102844412705217835
102844235752963525
102844412712164667
102844412705545516
102844341912220311
102844341907370644
102844235749424575
102844212429419722
102844294669568199
102844212431779031
102844294666422466
102844224146472059
102844212428895431
102844212429747404
10284423574870367

In [23]:
with open('./2features_ent_emd+채팅_undersampling/lstm_feature_train3.json','a') as f:
    json.dump(result,f)

In [18]:
len(result['102844212431058132'][0])

23

In [4]:
        with open('../../data/audio_entropy_emd.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [5]:
audio_result

{'102844212428895431': array([-0.00281934,  0.00271739, -0.00069991, ...,  0.0188171 ,
         0.01678068,  0.0132905 ]),
 '102844212429092040': array([-0.00083577,  0.00085681,  0.00038645, ..., -0.00065385,
         0.00362957, -0.00391633]),
 '102844212429419722': array([-0.00063095,  0.00032302,  0.00058857, ..., -0.01253491,
        -0.0113009 , -0.01018886]),
 '102844212429550795': array([-0.00269244, -0.00108717,  0.00562366, ...,  0.00819015,
        -0.00276938, -0.00373554]),
 '102844212429747404': array([ 0.00608276,  0.0073444 ,  0.00791687, ..., -0.00603843,
        -0.00274157, -0.00177838]),
 '102844212429944013': array([0.00396402, 0.00510628, 0.00383899, ..., 0.00909347, 0.00748892,
        0.00483823]),
 '102844212430075086': array([0.00899328, 0.01381409, 0.01318098, ..., 0.49989855, 0.49134774,
        0.46209633]),
 '102844212430402768': array([-0.00259072, -0.00279067,  0.00277035, ...,  0.02737126,
        -0.0274116 , -0.02732145]),
 '102844212430599377': array

In [6]:
            with open('../../data/chat_feature_test.json',"rb") as f1:  
                chat_result=json.load(f1)

In [9]:
chat_result['102844212431058132'][:100]

[[0.0025239158421754837,
  -0.004152143374085426,
  0.021946078166365623,
  -0.06446272879838943,
  -0.03039204701781273,
  -0.03674256429076195,
  -0.06599866598844528,
  -0.08853255957365036,
  0.008685160428285599,
  -0.005040775518864393,
  0.041659146547317505,
  -0.007091965992003679,
  0.017812082543969154,
  0.012573298998177052,
  -0.024586835876107216,
  0.06607580929994583,
  -0.04842275381088257,
  -0.05190718173980713,
  -0.08685580641031265,
  -0.050701968371868134,
  0.034265052527189255,
  -0.05727192759513855,
  0.015244542621076107,
  0.016512781381607056,
  0.00387037405744195,
  0.019040750339627266,
  -0.030914688482880592,
  -0.0017324202926829457,
  -0.07704826444387436,
  -0.035020336508750916,
  0.01727118343114853,
  0.06466113775968552,
  0.02081640064716339,
  -0.05807866156101227,
  -0.03312738239765167,
  0.05284546688199043,
  -0.0671476274728775,
  0.01667635701596737,
  0.019857829436659813,
  -0.04281878471374512,
  0.06510315835475922,
  0.00104659737